In [100]:
import pandas as pd

# Import file csv to dataframe
prices = pd.read_csv('Downloads/prices.csv')
sales = pd.read_csv('Downloads/sales.csv')

# Change datatype from object to datetime
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])

# Sort sales value by ordered_at
sales_sorted = sales.sort_values('ordered_at')
prices_sorted = prices.sort_values('updated_at')
                                   
# Merge 2 table using merge_asof
merge = pd.merge_asof(
     sales_sorted, prices_sorted,
    left_on='ordered_at',
    right_on='updated_at',
    by='product_id',
    direction='nearest'
)

# Sort by product_id
merge_sorted = merge.sort_values(['product_id', 'ordered_at'], ascending = False)

#define calculate_price function and create column 'price'
def calculate_price(row):
    if row['ordered_at'] >= row['updated_at']:
        return row['new_price']
    else:
        return row['old_price']
merge_sorted['price'] = merge.apply(calculate_price, axis=1)

#add column 'revenue'
merge_sorted['revenue'] = merge_sorted['price'] * merge_sorted['quantity_ordered']

#calculate total revene of each product at each price
merge_groupby = merge_sorted.groupby(['product_id','price'])['revenue'].sum().reset_index()
merge_groupby = merge_groupby.rename(columns={'revenue': 'Total_Revenue'})

#print total revene of each product at each price
print(merge_groupby)

    product_id   price  Total_Revenue
0           64  239000         956000
1      3954203   57500          57500
2      3954203   60000         180000
3      3954203   64000         640000
4      3998909   15500          15500
5      3998909   16500         231000
6      3998909   17000          34000
7      4085861   52000        1040000
8      4085861   53500        2140000
9      4085861   58000        2204000
10     4085861   60000         180000
11     4085861   62500        1812500
12     4085861   67000         871000
